# About this kernel
![](https://d1uzk9o9cg136f.cloudfront.net/f/16783342/rc/2019/10/11/0e310b5fae494ff287cf769a37c9382672f255f6_xlarge.jpg)



Recently there was historical biggest typhoon in Japan(named Hagibis), and made devastiting effect,
and I wondered if the number of typhoons is increasing.
so made some analysis and want to share you how Japanese typhoon trend changes.

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math

# Data of 2019 isn't filled, so this time I dropped it

In [ ]:
for i in range(len(os.listdir('../input'))-1):
    year = str(2001+i)
    name = 'table' + year + '.csv'
    if i == 0:
        data = pd.read_csv('../input/'+name, encoding='shift-jis')
    else:
        data = pd.concat([data,pd.read_csv('../input/'+name, encoding='shift-jis')],axis=0)
data = data.rename(columns = {'年':'year','月':'month','日':'day','時（UTC）':'hour','台風番号':'typhoon number','台風名':'typhoon name','階級':'grade','緯度':'longitude','経度':'latitude','中心気圧':'center pressure','最大風速':'max wind speed','50KT長径方向':'50KT major axis direction','50KT長径':'50KT major axis diameter','50KT短径':'50KT minor axis diameter','30KT長径方向':'30KT major axis direction','30KT長径':'30KT major axis diameter','30KT短径':'30KT minor axis diameter','上陸':'landing'})

In [ ]:
data.head()

# Simple Data Description


![](https://classmethod.jp/wp-content/uploads/2016/10/thu_logo_kishochou.png)


This data is from Japanese Official Metrological Agency.

You can get this data from here:
https://www.data.jma.go.jp/fcd/yoho/typhoon/position_table

# year, month, day, hour

time based features of typhoons

In [ ]:
for col in ['year','month','day','hour']:
    print(col)
    print(np.sort(data[col].unique()))

# typhoon number, name

identification of each typhoon

In [ ]:
print(data['typhoon number'].unique().shape)
print(data['typhoon name'].unique().shape)

# grade

1 : no typhoon (not included in this dataset)

2 : typhoon which max wind speed < 17m/s (34 knot)

3 : typhoon which 17m/s (34 knot) <= max wind speed < 25m/s (48 knot)

4 : typhoon which 25m/s (48 knot) <= max wind speed < 33m/s (64 knot)

5 : typhoon which 33m/s (64 knot) <= max wind speed

6 : temperate cyclone

7 : center of typhoon is out of north longitude 0-60°, east latitude 100-180°

In [ ]:
data['grade'].value_counts()

# longitude & latitude

geological point of typhoon

Let's plot examples of 2d courses of typhoons

In [ ]:
name = 'CIMARON'
num = 101
x = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['latitude'].values
y = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['longitude'].values
fig = plt.figure(facecolor="skyblue")
plt.plot(x,y,'darkorchid')
plt.hlines([10,20,30],110,140,"black", linestyles='dashed',linewidth=1)
plt.vlines([110,120,130,140],10,30,"black", linestyles='dashed',linewidth=1)
plt.show()

It corresponds to the mercator plot of this,


![](https://www.data.jma.go.jp/fcd/yoho/data/typhoon/T0101.png)

# Course of typhoon based on longitude/latitude

In [ ]:
example_num = 1
jp_x = [145,143,141,142,140,136,133,131,129,133,137,139,140,141,142,145]
jp_y = [44,41,42,40,35,34,33,30,33,36,37,38,41,42,45,44]

for name in data['typhoon name'].unique()[:example_num]:
    print(name)
    for num in data[data['typhoon name']==name]['typhoon number'].unique():
        
        x = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['latitude'].values
        y = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['longitude'].values
        
        plt.figure(facecolor="white")
        plt.axhspan(10,50,0.04,0.96,color='skyblue')
        plt.fill(jp_x,jp_y,color="g",alpha=0.5)
        plt.plot(x,y,'darkorchid')
        plt.xlabel('longitude')
        plt.ylabel('latitude')
        plt.hlines([10,20,30,40,50],110,160,"black", linestyles='dashed',linewidth=1)
        plt.vlines([110,120,130,140,150,160],10,50,"black", linestyles='dashed',linewidth=1)
        plt.show()

In [ ]:
print(np.sort(data['latitude'].values))
print(np.sort(data['longitude'].values))

# center pressure

air pressure at the center of typhoon

In [ ]:
np.sort(data['center pressure'].unique())

# max wind speed

In [ ]:
data['max wind speed'].unique()

# 50/30 knot major/minor axis direction/diameter


Japanese storm range is defined by a circle which:

1. center is shifted (50 knot major axis diameter - 50 knot minor axis diameter) to the direction of major axis diameter

2. direction has 8 types, 1:NE, 2:E, 3:SE, 4:S, 5:SW, 6:W, 7:NW, 8:N 

(N:North, S:South, E:East,W:West), and 9(wind direction is concentric to the center of typhoon)

Let's plot it with formor 2d course of typhoon (it's littlebit biased)

In [ ]:
example_num = 1
jp_x = [145,143,141,142,140,136,133,131,129,133,137,139,140,141,142,145]
jp_y = [44,41,42,40,35,34,33,30,33,36,37,38,41,42,45,44]

for name in data['typhoon name'].unique()[:example_num]:
    print(name)
    for num in data[data['typhoon name']==name]['typhoon number'].unique():
        print(num)
        x = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['latitude'].values
        y = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['longitude'].values
        
        major_diameter_50 = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['50KT major axis diameter'].values
        major_direction_50 = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['50KT major axis direction'].values
        minor_diameter_50 = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['50KT minor axis diameter'].values
        major_diameter_30 = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['30KT major axis diameter'].values
        major_direction_30 = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['30KT major axis direction'].values
        minor_diameter_30 = data[(data['typhoon name']==name)&(data['typhoon number']==num)]['30KT minor axis diameter'].values
        shifted_x = x + ((major_diameter_50-minor_diameter_50)/2)*np.sin((np.pi/4)*major_direction_50)*(1.852/111)
        shifted_y = y + ((major_diameter_50-minor_diameter_50)/2)*np.cos((np.pi/4)*major_direction_50)*(1.852/111)
        diameter = ((major_diameter_50+minor_diameter_50)/2)*(1.852/111)
        diameter_30 = ((major_diameter_30+minor_diameter_30)/2)*(1.852/111)
        fig = plt.figure(facecolor="white")
        plt.axhspan(10,50,0.04,0.96,color='skyblue')
        plt.fill(jp_x,jp_y,color="g",alpha=0.5)
        # 50 knot storm range
        for i in range(len(shifted_x)):
            circle_x = []
            circle_y = []
            for j in np.linspace(-180,180,360):
                circle_x.append(shifted_x[i]+diameter[i]*math.cos(math.radians(j)))
                circle_y.append(shifted_y[i]+diameter[i]*math.sin(math.radians(j)))
            plt.plot(circle_x,circle_y,'red')
        # 30 knot storm range
        for i in range(len(shifted_x)):
            circle_x = []
            circle_y = []
            for j in np.linspace(-180,180,360):
                circle_x.append(shifted_x[i]+diameter_30[i]*math.cos(math.radians(j)))
                circle_y.append(shifted_y[i]+diameter_30[i]*math.sin(math.radians(j)))
            plt.plot(circle_x,circle_y,'green',linewidth=0.5)
        plt.plot(x,y)
        plt.plot(jp_x,jp_y,'black')
        plt.plot(shifted_x,shifted_y,'red')
        plt.xlabel('longitude')
        plt.ylabel('latitude')
        plt.hlines([10,20,30,40,50],110,160,"black", linestyles='dashed',linewidth=1)
        plt.vlines([110,120,130,140,150,160],10,50,"black", linestyles='dashed',linewidth=1)
        plt.show()

# Then Let's watch about change of the numbers of typhoons

In [ ]:
print('Showing numbers of typhoons for each year')
numbers = {}
for year in data['year'].unique():
    numbers[year] = len(data[data['year']==year]['typhoon number'].unique())
    #print(str(year)+' : ',len(data[data['year']==year]['typhoon number'].unique()))
plt.figure(facecolor='gray')
plt.plot(list(numbers.keys()),list(numbers.values()),'black')
plt.show()
print('correlation coefficient : '+str(np.corrcoef(list(numbers.keys()),list(numbers.values()))[0][1]))

year & number of typhoons have 0.256 correlation 
I felt it's smaller than I thought,
it may be because I'm a teenager and compared only 2010〜 typhoons, Let's check it out

In [ ]:
numbers = {}
for year in data['year'].unique()[9:]:
    numbers[year] = len(data[data['year']==year]['typhoon number'].unique())
    #print(str(year)+' : ',len(data[data['year']==year]['typhoon number'].unique()))
plt.figure(facecolor='gray')
plt.plot(list(numbers.keys()),list(numbers.values()))
plt.show()
print('correlation coefficient : '+str(np.corrcoef(list(numbers.keys()),list(numbers.values()))[0][1]))

that's why I thought these days numbers of typhoons is increasing.

# Next Let's see how the size of typhoon changed
It is an important features directly related to how large does the typhoon have its effects.

In [ ]:
print('Showing average size of typhoons for each year')
print('Circle size reveals typhoon size')
sizes = {}
for year in data['year'].unique():
    sizes[year] = np.mean((data[data['year']==year]['50KT major axis diameter'].values + data[data['year']==year]['50KT minor axis diameter'].values)/2)
plt.figure(facecolor='gray')
plt.plot(list(sizes.keys()),list(sizes.values()),'black')
for i in range(len(sizes)):
    circle_x = []
    circle_y = []
    for j in np.linspace(-180,180,360):
        circle_x.append(list(sizes.keys())[i]+(list(sizes.values())[i]/50)*math.cos(math.radians(j)))
        circle_y.append(list(sizes.values())[i]+(list(sizes.values())[i]/30)*math.sin(math.radians(j)))
    plt.plot(circle_x,circle_y)
plt.show()
print('correlation coefficient : '+str(np.corrcoef(list(sizes.keys()),list(sizes.values()))[0][1]))

It says there is correlation -0.463 between year & average size of typhoon / year.
so how about max size of typhoon? 

In [ ]:
print('Showing max size of typhoons for each year')
sizes = {}
for year in data['year'].unique():
    sizes[year] = max((data[data['year']==year]['50KT major axis diameter'].values + data[data['year']==year]['50KT minor axis diameter'].values)/2)
plt.figure(facecolor='gray')
for i in range(len(sizes)):
    circle_x = []
    circle_y = []
    for j in np.linspace(-180,180,360):
        circle_x.append(list(sizes.keys())[i]+(list(sizes.values())[i]/300)*math.cos(math.radians(j)))
        circle_y.append(list(sizes.values())[i]+(list(sizes.values())[i]/40)*math.sin(math.radians(j)))
    plt.plot(circle_x,circle_y)
plt.plot(list(sizes.keys()),list(sizes.values()))
plt.show()
print('correlation coefficient : '+str(np.corrcoef(list(sizes.keys()),list(sizes.values()))[0][1]))

There is no correlation between year & max size of typhoon for each year

# So, how about wind speed?
It's also the main destructive part of typhoon

In [ ]:
print('Showing average max wind speed of typhoons for each year')
sizes = {}
for year in data['year'].unique():
    sizes[year] = np.mean(data[data['year']==year]['max wind speed'].values)
plt.figure(facecolor='gray')
plt.plot(list(sizes.keys()),list(sizes.values()),'black')
plt.show()
print('correlation coefficient : '+str(np.corrcoef(list(sizes.keys()),list(sizes.values()))[0][1]))

It also shows there is minus correlation, and tells us average max speed are relatively decreasing.

# Landing

In [ ]:
print('Showing numbers of landed typhoons for each year')
numbers = {}
for year in data['year'].unique():
    count = 0
    for num in data[data['year']==year]['typhoon number'].unique():
        if not (len(data[(data['year']==year)&(data['typhoon number']==num)]['landing'].unique())==1)&(data[(data['year']==year)&(data['typhoon number']==num)]['landing'].unique()[0]==0):
            count += 1
    numbers[year] = count
plt.figure(facecolor='gray')
plt.plot(list(numbers.keys()),list(numbers.values()))
plt.show()
print('correlation coefficient : '+str(np.corrcoef(list(numbers.keys()),list(numbers.values()))[0][1]))

there is clearly big correlation in these recent 10 years but totally it's not related

From this analysis, typhoons are clearly increasing these 10 years in Japan, but in 20 years there is not such a relationship, and average wind speed is decreasing.
Next time I'm gonna try to make some analysis about global warming.
If there are some advice please comment!

# Thank you for watching!